In [42]:
import psycopg2 as pg
import pandas as pd

In [43]:
import pandas.io.sql as psql

# Get work by WP

In [44]:
connection = pg.connect("host=192.168.0.20 port=5433 dbname=bregis_lis user=lis_admin password=Bre91s_Admin")
select_wp = """
select w."text" work_name
from lab.work as w,
	 lab.workplace as wp
where wp."text" = 'SysmexXS500i'
    and w.workplace_id = wp.id
"""
# dataframe_wp = psql.read_sql(select, connection)

cursor = connection.cursor()

cursor.execute(select_wp)
wp = cursor.fetchall()
works = [w[0] for w in wp]
works

[]

# Get Data

In [45]:
# connection = pg.connect("host=10.10.11.116 port=5433 dbname=bregis_lis user=lis_admin password=Bre91s_Admin")
select = """
select wr."text" result_name, wrv.*
from lab.work as w,
   lab.workresult as wr,
   lab.workresult_value as wrv
where w."text" = 'Определение РНК вируса гриппа В [DT96]'
    and wr.work_id = w.id
    and wrv.workresult_id = wr.id
"""
dataframe = psql.read_sql(select, connection)
# product_category = psql.read_sql_query('select * from product_category', connection)
dataframe['result_name'] = dataframe['result_name'].str.lower()
dataframe

C:\Users\BatrakovDR\AppData\Local\Temp\ipykernel_25988\7745141.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = psql.read_sql(select, connection)


,result_name,id,createby,createdate,updateby,updatedate,workresult_id,code,text,type_id,...,display_format_id,result_print_format_id,norm_set_id,hide_zero_value,input_order,display_order,sp_func,ext_code,status,group_num
0,рнк вируса гриппа в,5489,0,2024-02-12 13:20:51.069130,0,2024-02-12 13:20:51.069130,5052,2042,Качественный,2040,...,None,None,None,0,0,0,None,None,1,None


In [46]:
func_df = dataframe.dropna(subset=['func'])
func_df = func_df[['result_name', 'id', 'func', 'text']]
func_df

,result_name,id,func,text


In [47]:
df = dataframe.loc[(dataframe['result_name'].isin(func_df['result_name']))]
df

,result_name,id,createby,createdate,updateby,updatedate,workresult_id,code,text,type_id,...,display_format_id,result_print_format_id,norm_set_id,hide_zero_value,input_order,display_order,sp_func,ext_code,status,group_num


# Get values

In [48]:
ident_1 = '(числовой)'
ident_2 = '(анализатор)'

In [49]:
names = df['result_name'].str.replace(ident_1, '')
names

Series([], Name: result_name, dtype: object)

In [50]:
target_values = dataframe[dataframe['result_name'].str.contains(ident_1)]
analizator_values = dataframe[dataframe['result_name'].str.contains(ident_2)]

C:\Users\BatrakovDR\AppData\Local\Temp\ipykernel_25988\3204257614.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  target_values = dataframe[dataframe['result_name'].str.contains(ident_1)]
C:\Users\BatrakovDR\AppData\Local\Temp\ipykernel_25988\3204257614.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  analizator_values = dataframe[dataframe['result_name'].str.contains(ident_2)]


## Analizator values

In [51]:
analizator_values

,result_name,id,createby,createdate,updateby,updatedate,workresult_id,code,text,type_id,...,display_format_id,result_print_format_id,norm_set_id,hide_zero_value,input_order,display_order,sp_func,ext_code,status,group_num


In [52]:
analizator_values_without_ident = analizator_values['result_name'].str.replace(ident_2, '')

In [53]:
analizator_values.loc[:, ('result_name')] = analizator_values_without_ident
analizator_values

,result_name,id,createby,createdate,updateby,updatedate,workresult_id,code,text,type_id,...,display_format_id,result_print_format_id,norm_set_id,hide_zero_value,input_order,display_order,sp_func,ext_code,status,group_num


## Target values

In [54]:
target_values

,result_name,id,createby,createdate,updateby,updatedate,workresult_id,code,text,type_id,...,display_format_id,result_print_format_id,norm_set_id,hide_zero_value,input_order,display_order,sp_func,ext_code,status,group_num


In [55]:
target_values_without_ident = target_values['result_name'].str.replace(ident_1, '')
target_values_without_ident

Series([], Name: result_name, dtype: object)

In [56]:
target_values.loc[:, ('result_name')] = target_values_without_ident
target_values

,result_name,id,createby,createdate,updateby,updatedate,workresult_id,code,text,type_id,...,display_format_id,result_print_format_id,norm_set_id,hide_zero_value,input_order,display_order,sp_func,ext_code,status,group_num


# Changing formulas

In [57]:
target_names = target_values['result_name'].tolist()
target_names

[]

In [58]:
for name in target_names:
    idx_anal = analizator_values.loc[analizator_values['result_name'] == name]['id'].tolist()[0]
    target_func = target_values.loc[target_values['result_name'] == name]['func'].tolist()[0]
    # find all values in func
    i = 0
    value_indxs = []
    while i != -1:
        # print(target_func[sum(value_indxs)::])
        i = target_func[sum(value_indxs)+1::].find('#')
        value_indxs.append(i+1)
    # last element is bad
    value_indxs.pop(-1)
    # replace all values in func on new values
    for n in value_indxs:
        tf = target_func[:n+1+1:] + str(idx_anal) + target_func[n+1+1+4::]
        target_values.loc[target_values['result_name'] == name, 'func'] = tf

In [59]:
target_values['id']

Series([], Name: id, dtype: int64)

# Update

In [36]:
cursor = connection.cursor()
ind = target_values.index
for i in ind:
    v = target_values.loc[i]
    table_modification = f"""UPDATE lab.workresult_value as wrv SET func = '{str(v['func'])}' WHERE id = {int(v['id'])};"""
    # print(table_modification)
    try:
        cursor.execute(table_modification)
        connection.commit()
    except:
        print('Except!')
        cursor.execute("ROLLBACK")
        connection.commit()